In [5]:
import pandas as pd
import numpy as np 
import sys
sys.path.append('../')

def solution():
    
    #******************* CHANGE FILE PATH ****************#
    df_train=pd.read_csv('res/training/Pacific_train.csv')
    df_test=pd.read_csv('res/validation/Pacific_test.csv')
    #df_train=pd.read_csv(r'C:\01 - Self Learnings\AI ML\Edureka\Assignments\03 - ML -Assignment 2 - MultiClass Classifiers\Pacific_train.csv')
    #df_test=pd.read_csv(r'C:\01 - Self Learnings\AI ML\Edureka\Assignments\03 - ML -Assignment 2 - MultiClass Classifiers\Pacific_test.csv')
    #******************* CHANGE FILE PATH ****************#
    
    ########################################################################################################
    ###### Data analysis
    ########################################################################################################
    #print('Data types in columns:\n',df_train.dtypes,'\n\n\n')
    #print('Null Values analysis:\n',df_train.isnull().sum(),'\n\n\n')
    
    ########################################################################################################
    ###### Drop unwanted object columns
    ########################################################################################################
    df_train = df_train.drop(['ID','Name','Date','Time','Latitude','Longitude'], axis = 1)
    df_test = df_test.drop(['ID','Name','Date','Time','Latitude','Longitude'], axis = 1)
    #print(df_train.head())
    # Check pending datatypes
    #print('Data types in columns:\n',df_train.dtypes,'\n\n\n')
    
    ########################################################################################################
    ####### Label Encoding actual categorical data
    ########################################################################################################
    from sklearn.preprocessing import LabelEncoder
    le_event = LabelEncoder()
    df_train['Event'] = le_event.fit_transform(df_train['Event'])
    df_test['Event'] = le_event.transform(df_test['Event'])
    
    le_status = LabelEncoder()
    df_train['Status'] = le_event.fit_transform(df_train['Status'])
    df_test['Status'] = le_event.transform(df_test['Status'])
    
    #print(df_train.head())
    # Check pending datatypes
    #print('Data types in columns after removing unwanted columns and encoding:\n',df_train.dtypes,'\n\n\n')
    
    '''
    ########################################################################################################
    ###### Outlier Analysis and removal --> Not done because it removes more than 4000 rows
    ########################################################################################################
    old_shape = df_train.shape
    my_list = df_train.iloc[:,2:].columns.values.tolist()
    print(my_list)
    
    # Find Upper Bound and Lower Bound per column and store in a Dataframe - df_ub_lb
    df_ub_lb = pd.DataFrame(columns = ['Column Name', 'Upper Bound', 'Lower Bound'])
    row_num = 0
    for x in my_list:
        cur_col = df_train.loc[:,x]
        cur_col_sorted = cur_col.sort_values()
        iqr = cur_col_sorted.quantile(0.75) - cur_col_sorted.quantile(0.25)
        upper_bound = cur_col_sorted.quantile(0.75) + (1.5 * iqr)
        lower_bound = cur_col_sorted.quantile(0.25) - (1.5 * iqr)
        my_row = [x,upper_bound,lower_bound]
        df_ub_lb.loc[len(df_ub_lb)] = my_row
    display(df_ub_lb)
        
    #Remove Rows beyond upper bound and lower bound
    for x in my_list:
        # Use the lb and ub dataframe fro previous for loop to find ub and lb for the specific columns
        df_ub_lb_col = df_ub_lb.loc[df_ub_lb['Column Name'] == x]
        df_ub_lb_col = df_ub_lb_col.reset_index()
        lb = df_ub_lb_col.loc[0,'Lower Bound']
        ub = df_ub_lb_col.loc[0,'Upper Bound']
        # Remove data from the main dataframe
        df_train.drop(df_train[df_train[x] > ub].index, inplace = True)
        df_train.drop(df_train[df_train[x] < lb].index, inplace = True)
        
    new_shape = df_train.shape
    print('Shape of original dataframe: ', old_shape)
    print('Shape after removing outliers: ', new_shape)
    
    ########################################################################################################
    # Corelation Heat map --> Using this we see that all columns from Low Wind NE are strongly co-related
    # So we can take only 1 from these
    ########################################################################################################
    import seaborn as sns
    import matplotlib.pyplot as plt
    matrix = np.triu(df_train.corr())
    fig, ax = plt.subplots(figsize=(20,10))
    sns.heatmap(df_train.corr(), annot = True , vmin =-1 , vmax = 1, center = 0, cmap= 'coolwarm', 
                mask = matrix,linewidth=1, linecolor = 'black')
    '''
    
    ########################################################################################################
    #### Split data into X and y
    #### As per hint in question, we take features as Maximum Wind, Minimum Pressure, Low Wind NE 
    ########################################################################################################
    X_train = df_train.iloc[:,2:5]
    y_train = pd.DataFrame(df_train.iloc[:,1], columns = ['Status'])
    #print(X_train.head())
    #print(y_train.head())
    X_test = df_test.iloc[:,2:5]
    y_test = pd.DataFrame(df_test.iloc[:,1], columns = ['Status'])
    
    ########################################################################################################
    #### Apply Classification Models and their performance
    ########################################################################################################
    
    from sklearn.model_selection import cross_val_score
    from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score,precision_score
    import warnings
    warnings.filterwarnings('ignore')
    
    cross_val_dict = {}
    df_performance = pd.DataFrame(columns = ['Model','Accuracy','Recall','Precision','F1-Score'])
    
    # Decision Tree - Entropy
    from sklearn.tree import DecisionTreeClassifier
    dt_entropy = DecisionTreeClassifier(criterion='entropy')
    cross_val_dict['DecisionTreeClassifier'] = cross_val_score(dt_entropy, X_train, y_train, cv=10, scoring='accuracy').mean()
    dt_entropy.fit(X_train,y_train)
    y_pred_dt_entropy = dt_entropy.predict(X_test)
    # Find performance values for each model
    model = 'DecisionTreeClassifier'
    accuracy = round(accuracy_score(y_test, y_pred_dt_entropy),2)
    recall = recall_score(y_test, y_pred_dt_entropy, average = 'micro')
    precision = precision_score(y_test, y_pred_dt_entropy, average = 'micro')
    f1score = f1_score(y_test, y_pred_dt_entropy,average = 'micro')
    # Create performance matrics rows
    perf_row = [model,accuracy,recall,precision,f1score]
    df_performance.loc[len(df_performance)] = perf_row
    
    # Decision Gini - Gini
    dt_gini = DecisionTreeClassifier(criterion='gini')
    cross_val_dict['DecisionTreeClassifier_Gini'] = cross_val_score(dt_gini, X_train, y_train, cv=10, scoring='accuracy').mean()
    dt_gini.fit(X_train,y_train)
    y_pred_dt_gini = dt_gini.predict(X_test)
    # Find performance values for each model
    model = 'DecisionTreeClassifier_Gini'
    accuracy = round(accuracy_score(y_test, y_pred_dt_gini),2)
    recall = recall_score(y_test, y_pred_dt_gini,average = 'micro')
    precision = precision_score(y_test, y_pred_dt_gini,average = 'micro')
    f1score = f1_score(y_test, y_pred_dt_gini,average = 'micro')
    # Create performance matrics rows
    perf_row = [model,accuracy,recall,precision,f1score]
    df_performance.loc[len(df_performance)] = perf_row
    
    # Random Forest
    from sklearn.ensemble import RandomForestClassifier
    rf_model = RandomForestClassifier()
    cross_val_dict['RandomForestClassifier'] = cross_val_score(rf_model, X_train, y_train, cv=10, scoring='accuracy').mean()
    rf_model.fit(X_train,y_train)
    y_pred_rf = rf_model.predict(X_test)
    # Find performance values for each model
    model = 'RandomForestClassifier'
    accuracy = round(accuracy_score(y_test, y_pred_rf),2)
    recall = recall_score(y_test, y_pred_rf, average = 'micro')
    precision = precision_score(y_test, y_pred_rf, average = 'micro')
    f1score = f1_score(y_test, y_pred_rf,average = 'micro')
    # Create performance matrics rows
    perf_row = [model,accuracy,recall,precision,f1score]
    df_performance.loc[len(df_performance)] = perf_row
    
    # Naive Bayes
    from sklearn.naive_bayes import GaussianNB
    nb_model = GaussianNB()
    cross_val_dict['GaussianNB'] = cross_val_score(nb_model, X_train, y_train, cv=10, scoring='accuracy').mean()
    # Step 2 - Fit the Model
    nb_model.fit(X_train,y_train)
    # Step 3 - Predict from model
    y_pred_test_nb = nb_model.predict(X_test)
    # Find performance values for each model
    model = 'GaussianNB'
    accuracy = round(accuracy_score(y_test, y_pred_test_nb),2)
    recall = recall_score(y_test, y_pred_test_nb, average = 'micro')
    precision = precision_score(y_test, y_pred_test_nb,average = 'micro')
    f1score = f1_score(y_test, y_pred_test_nb,average = 'micro')
    # Create performance matrics rows
    perf_row = [model,accuracy,recall,precision,f1score]
    df_performance.loc[len(df_performance)] = perf_row
    
    #**** SVM Classifier Model
    from sklearn.svm import SVC
    # Step 1 - Call the model
    svc_model = SVC()
    cross_val_dict['SVC'] = cross_val_score(svc_model, X_train, y_train, cv=10, scoring='accuracy').mean()
    # Step 2 - Fit the model 
    svc_model.fit(X_train,y_train)
    # Step 3 -  Predict from model
    y_pred_test_svc = svc_model.predict(X_test)
    # Find performance values for each model
    model = 'SVC'
    accuracy = round(accuracy_score(y_test, y_pred_test_svc),2)
    recall = recall_score(y_test, y_pred_test_svc, average = 'micro')
    precision = precision_score(y_test, y_pred_test_svc, average = 'micro')
    f1score = f1_score(y_test, y_pred_test_svc, average = 'micro')
    # Create performance matrics rows
    perf_row = [model,accuracy,recall,precision,f1score]
    df_performance.loc[len(df_performance)] = perf_row
    
    df_best = df_performance.loc[df_performance['Accuracy'] == df_performance['Accuracy'].max()]
    df_output = df_best.iloc[:,0:2].T
    
    #print('Cross Val Scores: \n',cross_val_dict,'\n\n')
    #print('Performance of different models: \n',df_performance)
    
    
    #******************* CHANGE FILE PATH ****************#
    #df_output.to_csv(r'C:\01 - Self Learnings\AI ML\Edureka\Assignments\03 - ML -Assignment 2 - MultiClass Classifiers\output.csv', index = False, header = False)
    df_output.to_csv(r'output\output.csv', index = False, header = False)
    #******************* CHANGE FILE PATH ****************#

In [6]:
solution()